In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import pickle
import numpy as np
import os

In [14]:
files = open("metamorphosis.txt", 'r',encoding="latin1")
lines = []
for i in files:
    lines.append(i)

In [15]:
lines[0]

'The Project Gutenberg eBook of Metamorphosis\n'

In [20]:
data = ""
for i in lines:
    data = " ".join(lines)
data = data.replace('\n', '').replace('\r', '').replace(r'\uteff', '')

In [21]:
data[:400]

'The Project Gutenberg eBook of Metamorphosis      This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, y'

In [22]:
import string
translater = str.maketrans(string.punctuation, ' '*len(string.punctuation))
new_data = data.translate(translater)

In [24]:
new_data = new_data.translate(translater)
new_data[:400]

'The Project Gutenberg eBook of Metamorphosis      This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever  You may copy it  give it away or re use it under the terms of the Project Gutenberg License included with this ebook or online at www gutenberg org  If you are not located in the United States  y'

In [25]:
new = []
for i in data.split():
    if i not in new:
        new.append(i)
data = " ".join(new)

In [31]:
data

'The Project Gutenberg eBook of Metamorphosis This ebook is for the use anyone anywhere in United States and most other parts world at no cost with almost restrictions whatsoever. You may copy it, give it away or re-use under terms License included this online www.gutenberg.org. If you are not located States, will have to check laws country where before using eBook. *** a COPYRIGHTED Details Below. Please follow copyright guidelines file. Title: Author: Franz Kafka Translator: David Wyllie Release date: August 17, 2005 [eBook #5200] Most recently updated: April 28, 2021 Language: English START OF THE PROJECT GUTENBERG EBOOK METAMORPHOSIS by Translated I One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed his bed into horrible vermin. He lay on armour-like back, if lifted head little could see brown belly, slightly domed divided arches stiff sections. bedding was hardly able cover seemed ready slide off any moment. His many legs, pitifully thin compare

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [33]:
pickle.dump(tokenizer, open('tokenizer.pkl','wb'))

In [36]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [37]:
sequence_data[:10]

[57, 329, 5, 17, 30, 330, 8, 17, 58, 31]

In [38]:
vocab_size = len(tokenizer.word_index)+1

In [39]:
vocab_size

3246

In [41]:
sequences = []
for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)

In [42]:
len(sequences)

4773

In [44]:
sequences = np.array(sequences)

In [45]:
sequences[-2]

array([ 860, 3245])

In [46]:
x =[]
y = []
for i in sequences:
    x.append(i[0])
    y.append(i[1])
x = np.array(x)
y = np.array(y)

In [48]:
y = to_categorical(y, num_classes = vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [50]:
model = Sequential()

In [52]:
model.add(Embedding(vocab_size, 10, input_length = 1))
model.add(LSTM(units=1024, return_sequences=True))
model.add(LSTM(units=1024))
model.add(Dense(1024, activation='relu'))
model.add(Dense(vocab_size, activation = 'softmax'))

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             32460     
                                                                 
 lstm (LSTM)                 (None, 1, 1024)           4239360   
                                                                 
 lstm_1 (LSTM)               (None, 1024)              8392704   
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                                 
 dense_1 (Dense)             (None, 3246)              3327150   
                                                                 
Total params: 17,041,274
Trainable params: 17,041,274
Non-trainable params: 0
_________________________________________________________________


In [56]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard

checkpoints = ModelCheckpoint('model_nwp.h5', monitor='loss', verbose=1, save_best_only=True, mode='auto')
reduces = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr = 0.0001, verbose=1)
logdir = 'lognw'
tensorboard_vis = TensorBoard(log_dir= logdir)

In [58]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(learning_rate = 0.001), loss= 'categorical_crossentropy')

In [61]:
model.fit(x,y, epochs=75, callbacks = [checkpoints, reduces, tensorboard_vis])

Epoch 1/75
150/150 [==============================] - ETA: 0s - loss: 8.0974
Epoch 1: loss improved from inf to 8.09738, saving model to model_nwp.h5
150/150 [==============================] - 92s 548ms/step - loss: 8.0974 - lr: 0.0010
Epoch 2/75
150/150 [==============================] - ETA: 0s - loss: 8.0619
Epoch 2: loss improved from 8.09738 to 8.06194, saving model to model_nwp.h5
150/150 [==============================] - 68s 456ms/step - loss: 8.0619 - lr: 0.0010
Epoch 3/75
150/150 [==============================] - ETA: 0s - loss: 7.9348
Epoch 3: loss improved from 8.06194 to 7.93485, saving model to model_nwp.h5
150/150 [==============================] - 67s 447ms/step - loss: 7.9348 - lr: 0.0010
Epoch 4/75
150/150 [==============================] - ETA: 0s - loss: 7.5881
Epoch 4: loss improved from 7.93485 to 7.58805, saving model to model_nwp.h5
150/150 [==============================] - 67s 444ms/step - loss: 7.5881 - lr: 0.0010
Epoch 5/75
150/150 [========================

Epoch 35/75
150/150 [==============================] - ETA: 0s - loss: 2.0208
Epoch 35: loss improved from 2.08987 to 2.02084, saving model to model_nwp.h5
150/150 [==============================] - 84s 561ms/step - loss: 2.0208 - lr: 0.0010
Epoch 36/75
150/150 [==============================] - ETA: 0s - loss: 1.9753
Epoch 36: loss improved from 2.02084 to 1.97530, saving model to model_nwp.h5
150/150 [==============================] - 91s 610ms/step - loss: 1.9753 - lr: 0.0010
Epoch 37/75
150/150 [==============================] - ETA: 0s - loss: 1.9080
Epoch 37: loss improved from 1.97530 to 1.90802, saving model to model_nwp.h5
150/150 [==============================] - 90s 598ms/step - loss: 1.9080 - lr: 0.0010
Epoch 38/75
150/150 [==============================] - ETA: 0s - loss: 1.8470
Epoch 38: loss improved from 1.90802 to 1.84705, saving model to model_nwp.h5
150/150 [==============================] - 79s 528ms/step - loss: 1.8470 - lr: 0.0010
Epoch 39/75
150/150 [===========


Epoch 69: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
150/150 [==============================] - 65s 430ms/step - loss: 1.1222 - lr: 0.0010
Epoch 70/75
150/150 [==============================] - ETA: 0s - loss: 0.8452
Epoch 70: loss improved from 1.10528 to 0.84522, saving model to model_nwp.h5
150/150 [==============================] - 69s 457ms/step - loss: 0.8452 - lr: 2.0000e-04
Epoch 71/75
150/150 [==============================] - ETA: 0s - loss: 0.7454
Epoch 71: loss improved from 0.84522 to 0.74542, saving model to model_nwp.h5
150/150 [==============================] - 67s 447ms/step - loss: 0.7454 - lr: 2.0000e-04
Epoch 72/75
150/150 [==============================] - ETA: 0s - loss: 0.7165
Epoch 72: loss improved from 0.74542 to 0.71649, saving model to model_nwp.h5
150/150 [==============================] - 67s 445ms/step - loss: 0.7165 - lr: 2.0000e-04
Epoch 73/75
150/150 [==============================] - ETA: 0s - loss: 0.7023
Epoch 73: loss impr

In [62]:
%load_ext tensorboard
%tensorboard --logdir= './logsnw'

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-

In [66]:
def predictive_text_next(Text):
    sequence = tokenizer.texts_to_sequences([text])[0]
    sequence = np.array(sequence)
    preds = model.predict(sequence)
    p_class = np.argmax(preds)
    p_word = tokenizer.index_word[p_class]
    return p_word

In [69]:
text = input("enter text:")
text = text.split(' ')
text = text[-1]
text = ''.join(text)
predictive_text_next(text)

enter text:re-use it under the terms of the
1/1 [==============================] - 0s 28ms/step


'project'